In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Corner/GeoCS'

In [21]:
from pathlib import Path

from shapely import wkt
from pandas import read_csv
from geopandas import GeoDataFrame, points_from_xy

BASE_DIR = Path(BASE_DIR)

In [17]:
opz = read_csv(BASE_DIR.joinpath('ops_zone.csv'))
opz['ops_zone'] = opz['ops_zone'].apply(wkt.loads)
opz = GeoDataFrame(opz, geometry='ops_zone')
opz.sample()

,city_code,city_name,zone_id,zone,ops_zone
37,MLM,Morelia,230,Morelia,"MULTIPOLYGON (((-101.23843 19.68834, -101.2483..."


In [23]:
zipc = read_csv(BASE_DIR.joinpath('MX_zipcode.csv'))
zipc = GeoDataFrame(zipc, geometry=points_from_xy(zipc['lat'], zipc['lon']))
zipc.sample()

,Unnamed: 0,country_code,postal_code,place_name,state_name,state_code,province_name,province_code,community_name,community_code,lat,lon,accuracy,geometry
18010,18010,MX,29930,Belisario Domínguez,Chiapas,7,Yajalón,109,NaN,NaN,17.1856,-92.3258,3,POINT (17.186 -92.326)


In [34]:
# aux = opz.sjoin(zipc, how="left", op='within')

In [2]:
from pandas import read_csv

users = read_csv('/Users/efraflores/Desktop/Merchant funded promo UUIDs.csv')
users.sample()

,name,user_uuid,customer_uuid
270667,nvg-ssl-merchants_nl_mx_20220415_treatment,5fe1e287-6167-4ac4-90ec-0c544f4d6037,ca2a217b-b9d1-4685-982a-ed6b11f2b141


In [3]:
orders = read_csv('/Users/efraflores/Desktop/apr_orders.csv')
orders.sample()

,customer_uuid,user_id,order_id,order_datetime,shopper_store_branch_id,qty_products_ordered,qty_products_found,qty_products_replaced,total_receipt
393057,b117c7c9-fba4-4f64-9819-b47b5ccc9c23,30075507,56767175,2022-04-26T20:35:40.384515,3105,1,1,0,237.0


In [18]:
df = users.merge(orders, on='customer_uuid')
df = df[df['name']=='nvg-ssl-merchants_nl_mx_20220415_treatment'].copy()
df.head()

,name,user_uuid,customer_uuid,user_id,order_id,order_datetime,shopper_store_branch_id,qty_products_ordered,qty_products_found,qty_products_replaced,total_receipt
2,nvg-ssl-merchants_nl_mx_20220415_treatment,70454b8f-5ce5-4294-8b7d-c5743b5d8138,a9564fae-022f-43e6-a36a-dc22520c2e1a,30054225,56727049,2022-04-26T11:58:13.903596,33887,1,1,0,240.00
3,nvg-ssl-merchants_nl_mx_20220415_treatment,72fd3348-b9f3-467a-8985-6164973b4d59,bab50e9b-1dc0-40ac-970a-5a540a5f0dc3,29763346,56285111,2022-04-20T09:41:55.212035,26965,4,4,0,105.00
4,nvg-ssl-merchants_nl_mx_20220415_treatment,734ead2a-fb0e-4a80-987e-6c8585696af0,43f8287b-3cb8-4b82-b600-8cae25b300a4,29491215,56688104,2022-04-25T20:14:15.006521,11755,23,20,2,770.93
5,nvg-ssl-merchants_nl_mx_20220415_treatment,5049733a-0e3a-4c56-834b-ee6d8510c4a1,1797ebdd-f1e7-4ff6-9acc-c8a3e23c1b12,2627134,55191333,2022-04-04T17:49:12.521888,35393,11,10,0,526.45
6,nvg-ssl-merchants_nl_mx_20220415_treatment,5049733a-0e3a-4c56-834b-ee6d8510c4a1,1797ebdd-f1e7-4ff6-9acc-c8a3e23c1b12,2627134,55261517,2022-04-05T16:45:53.087955,35393,5,5,0,251.09


In [20]:
df.to_csv('/Users/efraflores/Desktop/apr_orders_users.csv', index=False)